In [ ]:
ifcs = []
n_structs = []
for iter in range(2, niters+1):
    iter_dir = iters_dir.joinpath(f'iter_{iter}')
    unitcell = read(iter_dir.joinpath('infile.ucposcar'), format='vasp')
    supercell = read(iter_dir.joinpath('infile.ssposcar'), format='vasp')
    ifc = tdp.parse_outfile_forceconstants(iter_dir.joinpath('converged_outfile.forceconstant'), unitcell, supercell)
    ifcs.append(ifc)
print(len(ifcs))
ifcs = np.array(ifcs)
max_diffs = [np.max(np.abs(ifcs[i] - ifcs[i-1]), axis=1) for i in range(1, len(ifcs)+1)]
Fig = plt.figure(figsize=(15,4))
plt.plot(nconfs[iter-1], max_diffs, '.')
plt.title('IFC convergence: max abs. error')
plt.ylabel('Error on the IFCs (eV/$\mathrm{\AA}^2$)')
plt.xlabel('Number of structures')
figpath = dir.joinpath(f'Convergence.png')
#plt.savefig(fname=figpath, bbox_inches='tight', dpi=600, format='png')


0.9799


0.9946463992227665

In [98]:

from pathlib import Path
from ase.io import read

def parse_outfile_forceconstants(filepath, unitcell, supercell):
    
    def find_index_in_unitcell(red_position, unitcell):
        scaled_positions = unitcell.get_scaled_positions()
        return np.argmin(np.linalg.norm(scaled_positions - red_position, axis=1))

    mat = supercell.get_cell() @ np.linalg.inv(unitcell.get_cell())
    filepath = Path(filepath)
    
    with open(filepath, 'r') as fl:
        lines = fl.readlines()
    lines = [x.split() for x in lines]
    nats_s = len(supercell)
    nats_u = len(unitcell)

    cell = unitcell.get_cell()
    positions = supercell.get_positions() # nats, 3
    positions_red = (np.linalg.inv(cell.T) @ positions.T).T
    atoms_tuples = []
    for i, atom in enumerate(supercell):
        frac_part = positions_red[i] % 1 # works for both positive and negative reduced coords!!! e.g. -1.3 % 1 = 0.7, not -0.3
        ind = find_index_in_unitcell(frac_part, unitcell)
        repetition_indices = [np.floor(positions_red[i][0]).astype(int), np.floor(positions_red[i][1]).astype(int),np.floor(positions_red[i][2]).astype(int)] # again, works for positive and negative numbers, np.floor(-1.3) = -2, not -1!!
        atoms_tuples.append((ind, *repetition_indices))
    # now atoms_tuples is a list of tuples (ind, R1, R2, R3), where ind the index of the atom in the unitcell and R1/2/3 are the component of the position of the repetition in reduced coordinates

    ifc = np.zeros((nats_u, nats_s, 3, 3))
    k = 2
    for i in range(nats_u):
        nns = int(lines[k][0]) # number of neighbours
        for n in range(nns):
            ci = k+1+5*n
            neigh_unit_ind = int(lines[ci][0])-1 # index of the neighbour in the unitcell
            vec_u = np.array([float(lines[ci+1][0]), float(lines[ci+1][1]), float(lines[ci+1][2])])
            
            # we need to change the basis of vec from the unitcell vectors to the supercell vectors
            # x^u = L x^s where L is P.T, where P is the mat_mult used to create the supercell from the unitcell
            # in principle we can compute x^s = L^-1 @ x^u, but np.solve is faster
            vec_s = np.linalg.solve(mat.T, vec_u)
            vec_s[np.abs(vec_s) < 1E-10] = 0
            vec_wrapped_s = vec_s % 1
            vec_wrapped_u = mat @ vec_wrapped_s
            vec_wrapped_u[np.abs(vec_wrapped_u) < 1E-10] = 0


            current_tuple = (neigh_unit_ind, vec_wrapped_u[0], vec_wrapped_u[1], vec_wrapped_u[2])
            j = atoms_tuples.index(current_tuple) # find the matching tuple, j is the index of the neighbour in the supercell
            tens = []
            tens.append([float(lines[ci+2][0]), float(lines[ci+2][1]), float(lines[ci+2][2])])
            tens.append([float(lines[ci+3][0]), float(lines[ci+3][1]), float(lines[ci+3][2])])
            tens.append([float(lines[ci+4][0]), float(lines[ci+4][1]), float(lines[ci+4][2])])
            tens = np.array(tens, dtype='float')
            ifc[i,j] += tens
        k += 1+5*nns

    return ifc

dir = Path('/Users/samuel/Work/developing/stdep_script/iterations/iter_4/ifc/')

fp = dir.joinpath('rc2_0/outfile.forceconstant')
unitcell = read(dir.joinpath('infiles/infile.ucposcar'), format='vasp')
supercell = read(dir.joinpath('infiles/infile.ssposcar'), format='vasp')
ifcs = parse_outfile_forceconstants(fp, unitcell, supercell)


In [84]:
np.array([-2.71002036e-18, -3.33333333e-01,  0.00000000e+00]) %1

array([0.        , 0.66666667, 0.        ])

In [71]:
-1/2%1

0.5

In [25]:
from ase.io import read
from ase.data import atomic_numbers as ase_atnums
from ase.data import chemical_symbols as ase_chemsym
import numpy as np

ats = read('/Users/samuel/Work/ML/MoSx/MD/melting/MD_88/w_mlmd.traj', index=':')[0]

syms = ats.get_chemical_symbols()
nums = [ase_atnums[x] for x in syms]
cutoff = 3.5
positions = ats.get_positions()
centers = ats[np.where(nums == 42)]
print(centers[0].position)
angles = []
for i, at in enumerate(centers):
    inds = np.where(np.linalg.norm(positions - [positions[i]]) < cutoff)
    for nb1 in inds:
        for nb2 in inds:
            if nb1 != nb2:
                if nums[nb1] == 32 and nums[nb2] == 32:
                    angles.append(np.arccos(positions[nb2] - positions[i], positions[nb1] - positions[i]))
        





[]


/var/folders/5t/cd9kht1x18l5ndqjk7tf68sr0000gn/T/ipykernel_59717/1372745914.py:12: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  centers = ats[np.where(nums == 42)]
/var/folders/5t/cd9kht1x18l5ndqjk7tf68sr0000gn/T/ipykernel_59717/1372745914.py:16: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  inds = np.where(np.linalg.norm(positions - [positions[i]]) < cutoff)
/var/folders/5t/cd9kht1x18l5ndqjk7tf68sr0000gn/T/ipykernel_59717/1372745914.py:19: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size

In [2]:
def a(c):
    print(c)
a()


TypeError: a() missing 1 required positional argument: 'c'